In [1]:
import pandas as pd
from sklearn.cluster import KMeans

df = pd.read_csv('../데이터/VIP_churnscoreO_11460_826.csv')

FileNotFoundError: [Errno 2] No such file or directory: '../데이터/VIP_churnscoreO_11460_826.csv'

In [ ]:
X = df[['Churn_Score']]

# KMeans 모델 생성 (군집 3개)
# random_state를 고정하면 돌릴 때마다 똑같은 결과가 나와서 좋아.
kmeans = KMeans(n_clusters=3, random_state=42)

# 학습 및 예측 (군집 번호 0, 1, 2 생성)
df['cluster_label'] = kmeans.fit_predict(X)

# 각 군집별로 데이터가 몇 개씩 들어갔는지 확인해볼까?
print("\n군집별 데이터 개수:")
print(df['cluster_label'].value_counts().sort_index())


군집별 데이터 개수:
cluster_label
0    3588
1    6630
2    1242
Name: count, dtype: int64


In [ ]:
# Target 변수 생성 함수 정의
def create_target(label):
    if label == 0:
        return 1  # 이탈 (군집 0)
    else:
        return 0  # 잔존 (군집 1, 2)

# apply 함수를 써서 Target 컬럼 만들기
df['Target'] = df['cluster_label'].apply(create_target)

# 잘 만들어졌는지 결과 확인!
print("\n생성된 Target 변수 분포:")
print(df['Target'].value_counts())

# 최종 데이터 확인
display(df[['Churn_Score', 'cluster_label', 'Target']].head(10))


생성된 Target 변수 분포:
Target
0    7872
1    3588
Name: count, dtype: int64


,Churn_Score,cluster_label,Target
0,4.2,1,0
1,13.5,0,1
2,11.4,0,1
3,7.2,1,0
4,2.1,1,0
5,2.6,1,0
6,7.7,1,0
7,8.9,0,1
8,4.7,1,0
9,13.8,0,1


In [ ]:
# 2,000개 데이터를 무작위로 추출해 (random_state를 고정하면 결과가 일정해!)
if len(df) > 2000:
    sampled_df = df.sample(n=2000, random_state=42).reset_index(drop=True)
    print(f"전체 {len(df)}개 데이터 중 2000개를 무작위 추출했어!")
else:
    sampled_df = df.copy()
    print("데이터가 2000개보다 적어서 전체 데이터를 사용할게!")

In [ ]:
import xgboost as xgb
import shap
import pandas as pd
import numpy as np
from statsmodels.stats.outliers_influence import variance_inflation_factor

# --- [1. 데이터 준비] ---
# 분석에 불필요한 컬럼 제거 (ID, 날짜, 타겟, 군집 등)
cols_to_drop = ['발급회원번호', '기준년월', 'Target', 'cluster_label', 'Churn_Score']
# 실제 데이터프레임에 있는 컬럼만 골라서 드롭 (에러 방지)
drop_cols = [c for c in cols_to_drop if c in sampled_df.columns]

X = sampled_df.drop(columns=drop_cols)
X = X.select_dtypes(include=[np.number]) # 숫자형 데이터만 선택
y = sampled_df['Target']

# --- [2. XGBoost 모델 학습] ---
model = xgb.XGBClassifier(
    n_estimators=100,
    learning_rate=0.1,
    max_depth=5,
    random_state=42,
    use_label_encoder=False,
    eval_metric='logloss'
)
model.fit(X, y)
print("XGBoost 모델 학습 완료!")

# --- [3. SHAP 중요도(Mean |SHAP|) 계산] ---
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X)

# 각 변수별 평균 절대값(Mean Absolute SHAP) 계산 -> 변수 중요도
shap_importance = np.abs(shap_values).mean(axis=0)

# --- [4. VIF 지수 계산] ---
# 결측치는 평균으로 채워서 계산 (VIF 오류 방지)
X_vif = X.fillna(X.mean())

# 모든 컬럼의 VIF 계산
vif_list = [variance_inflation_factor(X_vif.values, i) for i in range(len(X_vif.columns))]

# --- [5. 데이터프레임 통합 및 정렬] ---
result_df = pd.DataFrame({
    'feature': X.columns,
    'shap_importance': shap_importance,
    'vif': vif_list
})

# SHAP 중요도 기준으로 내림차순 정렬
result_df = result_df.sort_values(by='shap_importance', ascending=False)

/usr/local/Caskroom/miniconda/base/envs/myenv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/usr/local/Caskroom/miniconda/base/envs/myenv/lib/python3.13/site-packages/xgboost/training.py:199: UserWarning: [16:46:24] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1764148644238/work/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


XGBoost 모델 학습 완료!


/usr/local/Caskroom/miniconda/base/envs/myenv/lib/python3.13/site-packages/statsmodels/stats/outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)
/usr/local/Caskroom/miniconda/base/envs/myenv/lib/python3.13/site-packages/statsmodels/regression/linear_model.py:1782: RuntimeWarning: invalid value encountered in scalar divide
  return 1 - self.ssr/self.centered_tss


KeyboardInterrupt: 